In [ ]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [ ]:
class MobilePhone:
    def __init__(self, model, price, antutu_score, dxomark, ppi, battery_capacity):
        self.model = model
        self.price = price
        self.antutu_score = antutu_score
        self.dxomark = dxomark
        self.ppi = ppi
        self.battery_capacity = battery_capacity

    def __str__(self):
        return f"Model: {self.model}, Price ($): {self.price}, Antutu Score: {self.antutu_score}, DXOMARK: {self.dxomark}, PPI: {self.ppi}, Battery Capacity (mAh): {self.battery_capacity}"

    def __eq__(self, other):
        if not isinstance(other, MobilePhone):
            return NotImplemented
        return (self.model == other.model and
                self.price == other.price and
                self.antutu_score == other.antutu_score and
                self.dxomark == other.dxomark and
                self.ppi == other.ppi and
                self.battery_capacity == other.battery_capacity)


In [ ]:
#wczytaj smartfony i utwórz liste z objektami klasy MobilePhone
data = pd.read_csv("smartfony.csv")
phones = []
phones = [MobilePhone(row['Model'], int(row['Cena ($)']), int(row['Antutu Score']), 
          int(row['DXOMARK']), int(row['ppi']), int(row['pojemność baterii (mAh)'])) for index, row in data.iterrows()]



In [ ]:
#wyswietl telefony
for phone in phones:
    print(phone)

In [ ]:
class Criterion:
    def __init__(self, name, type, weight, indiff, pref, veto):
        self.name = name
        self.type = type
        self.weight = weight
        self.indiff = indiff
        self.pref = pref
        self.veto = veto

# Initialize criteria
criteria = [
    Criterion("price", "cost", 2, 50, 200, 600),
    Criterion("antutu", "gain", 4, 100000, 300000, 500000),
    Criterion("dxomark", "gain", 4, 10, 20, 40),
    Criterion("dpi", "gain", 2, 20, 50, 100),
    Criterion("battery", "gain", 3, 400, 1200, 2000)
]

ELECTRE

In [ ]:
# Creating the matrix of concordance
concordance = np.ones((len(phones), len(phones)))

# The mapping from phone attributes to the criteria list indices
attribute_mapping = {
    "price": lambda x: x.price,
    "antutu": lambda x: x.antutu_score,
    "dxomark": lambda x: x.dxomark,
    "dpi": lambda x: x.ppi,
    "battery": lambda x: x.battery_capacity
}

for i, phone1 in enumerate(phones):
    for j, phone2 in enumerate(phones):
        cis = []
        dis = []
        for criterion in criteria:
            value1 = attribute_mapping[criterion.name](phone1)
            value2 = attribute_mapping[criterion.name](phone2)
            
            if criterion.type == "gain":
                if value1 - value2 >= -criterion.indiff:
                    cis.append(1)
                elif value1 - value2 < -criterion.pref:
                    cis.append(0)
                else:
                    cis.append((criterion.pref - (value2 - value1)) / (criterion.pref - criterion.indiff))
                
                if value1 - value2 <= -criterion.veto:
                    dis.append(1)
                elif value1 - value2 >= -criterion.pref:
                    dis.append(0)
                else:
                    dis.append(((value2 - value1) - criterion.pref) / (criterion.veto - criterion.pref))

            else:  # Assuming 'cost' type here
                if value1 - value2 <= criterion.indiff:
                    cis.append(1)
                elif value1 - value2 > criterion.pref:
                    cis.append(0)
                else:
                    cis.append((criterion.pref - (value1 - value2)) / (criterion.pref - criterion.indiff))

                if value1 - value2 >= criterion.veto:
                    dis.append(1)
                elif value1 - value2 <= criterion.pref:
                    dis.append(0)
                else:
                    dis.append((criterion.veto - (value1 - value2)) / (criterion.veto - criterion.pref))

        c = sum(ci * criterion.weight for ci, criterion in zip(cis, criteria)) / sum(criterion.weight for criterion in criteria)
        sigma = c

        for d in dis:
            if d > c:
                sigma *= (1 - d) / (1 - c)
        
        concordance[i][j] = round(sigma, 2)

print(concordance)

In [ ]:
def descending_destilation(matrix, variants, upper_treshold=None, depth=0):
    ans = []
    depth += 1

    while True:
        n = matrix.shape[0]
        matrix_copy = deepcopy(matrix)

        if upper_treshold is None:
            mask = np.eye(n, dtype=bool)
            masked_arr = np.ma.masked_array(matrix, mask)
            upper_treshold = np.max(masked_arr)

        if upper_treshold == 0:
            if len(ans) == 1:
                return ans
            else:
                ans.extend(variants) 
                return ans
            
        if len(variants) == 0:
            return ans
        if len(variants) == 1:
            ans.extend(variants) 
            return ans
        
        s = -0.15 * upper_treshold + 0.3
        lower_treshold = np.max(matrix[matrix < upper_treshold - s]) if np.any(matrix < upper_treshold - s) else 0

        for i in range(n):
            for j in range(n):
                if matrix[i][j] > lower_treshold and matrix[i][j] > matrix[j][i] + s:
                    continue
                else:
                    matrix_copy[i][j] = 0

        strength = np.array([np.count_nonzero(row) for row in matrix_copy])
        weakness = np.array([np.count_nonzero(column) for column in matrix_copy.T])
        quality = strength - weakness

        if np.count_nonzero(quality == np.max(quality)) == 1:
            max_index = np.argmax(quality)
            ans.append(variants[max_index])
            variants.pop(max_index)
            matrix = np.delete(matrix, max_index, axis=0)
            matrix = np.delete(matrix, max_index, axis=1)
        else:
            max_indices = np.where(quality == np.max(quality))[0].tolist()
            internal_matrix = deepcopy(matrix_copy[max_indices][:, max_indices])
            internal_variants = [variants[i] for i in max_indices]

            internal_ans = descending_destilation(internal_matrix, internal_variants, lower_treshold, depth)
            internal_indices = []
            print("Incomparability!")
            for phone in internal_ans:
                print(phone)
            for variant in internal_ans:
                try:
                    internal_indices.append(variants.index(variant))
                except ValueError:
                    continue  # Skip if variant not found

            internal_indices = sorted(internal_indices, reverse=True)
            for internal_index in internal_indices:
                variants.pop(internal_index)

            matrix = np.delete(matrix, np.array(internal_indices), axis=0)
            matrix = np.delete(matrix, np.array(internal_indices), axis=1)

            ans.extend(internal_ans) 

        if matrix.shape[0] == 0:
            return ans


variants = deepcopy(phones)

des_dest = descending_destilation(concordance, variants)

In [ ]:
for phone in des_dest:
    print(phone)

Wstępująca destylacja

In [ ]:
#wczytaj smartfony i utwórz liste z objektami klasy MobilePhone
data = pd.read_csv("smartfony.csv")
phones = []
phones = [MobilePhone(row['Model'], int(row['Cena ($)']), int(row['Antutu Score']), 
          int(row['DXOMARK']), int(row['ppi']), int(row['pojemność baterii (mAh)'])) for index, row in data.iterrows()]

In [ ]:
# Creating the matrix of concordance
concordance_matrix = np.ones((len(phones), len(phones)))

# The mapping from phone attributes to the criteria list indices
attribute_mapping = {
    "price": lambda x: x.price,
    "antutu": lambda x: x.antutu_score,
    "dxomark": lambda x: x.dxomark,
    "dpi": lambda x: x.ppi,
    "battery": lambda x: x.battery_capacity
}

for i, phone1 in enumerate(phones):
    for j, phone2 in enumerate(phones):
        cis = []
        dis = []
        for criterion in criteria:
            value1 = attribute_mapping[criterion.name](phone1)
            value2 = attribute_mapping[criterion.name](phone2)
            
            if criterion.type == "gain":
                if value1 - value2 >= -criterion.indiff:
                    cis.append(1)
                elif value1 - value2 < -criterion.pref:
                    cis.append(0)
                else:
                    cis.append((criterion.pref - (value2 - value1)) / (criterion.pref - criterion.indiff))
                
                if value1 - value2 <= -criterion.veto:
                    dis.append(1)
                elif value1 - value2 >= -criterion.pref:
                    dis.append(0)
                else:
                    dis.append(((value2 - value1) - criterion.pref) / (criterion.veto - criterion.pref))

            else:  # Assuming 'cost' type here
                if value1 - value2 <= criterion.indiff:
                    cis.append(1)
                elif value1 - value2 > criterion.pref:
                    cis.append(0)
                else:
                    cis.append((criterion.pref - (value1 - value2)) / (criterion.pref - criterion.indiff))

                if value1 - value2 >= criterion.veto:
                    dis.append(1)
                elif value1 - value2 <= criterion.pref:
                    dis.append(0)
                else:
                    dis.append((criterion.veto - (value1 - value2)) / (criterion.veto - criterion.pref))

        c = sum(ci * criterion.weight for ci, criterion in zip(cis, criteria)) / sum(criterion.weight for criterion in criteria)
        sigma = c

        for d in dis:
            if d > c:
                if c == 1:
                    print("c = 1")
                sigma *= (1 - d) / (1 - c)
        
        concordance_matrix[i][j] = round(sigma, 2)

print(concordance_matrix)

In [ ]:
def ascending_destilation(matrix, variants, upper_treshold=None, depth=0):
    ans = []
    depth += 1

    while True:
        n = matrix.shape[0]
        matrix_copy = deepcopy(matrix)

        if upper_treshold is None:
            mask = np.eye(n, dtype=bool)
            masked_arr = np.ma.masked_array(matrix, mask)
            upper_treshold = np.max(masked_arr)

        if upper_treshold == 0:
            if len(ans) == 1:
                return ans
            else:
                ans.extend(variants) 
                return ans
            
        if len(variants) == 0:
            return ans
        if len(variants) == 1:
            ans.extend(variants) 
            return ans
        
        s = -0.15 * upper_treshold + 0.3
        lower_treshold = np.max(matrix[matrix < upper_treshold - s]) if np.any(matrix < upper_treshold - s) else 0

        for i in range(n):
            for j in range(n):
                if matrix[i][j] > lower_treshold and matrix[i][j] > matrix[j][i] + s:
                    continue
                else:
                    matrix_copy[i][j] = 0

        strength = np.array([np.count_nonzero(row) for row in matrix_copy])
        weakness = np.array([np.count_nonzero(column) for column in matrix_copy.T])
        quality = strength - weakness

        if np.count_nonzero(quality == np.min(quality)) == 1:
            min_index = np.argmin(quality)
            ans.insert(0, variants[min_index])
            variants.pop(min_index)
            matrix = np.delete(matrix, min_index, axis=0)
            matrix = np.delete(matrix, min_index, axis=1)
        else:
            min_indices = np.where(quality == np.min(quality))[0].tolist()
            internal_matrix = deepcopy(matrix_copy[min_indices][:, min_indices])
            internal_variants = [variants[i] for i in min_indices]

            internal_ans = ascending_destilation(internal_matrix, internal_variants, lower_treshold, depth)
            internal_indices = []
            print("Incomparability!")
            for phone in internal_ans:
                print(phone)
            for variant in internal_ans:
                try:
                    internal_indices.insert(0,variants.index(variant))
                except ValueError:
                    continue  # Skip if variant not found

            internal_indices = sorted(internal_indices, reverse=True)
            for internal_index in internal_indices:
                variants.pop(internal_index)

            matrix = np.delete(matrix, np.array(internal_indices), axis=0)
            matrix = np.delete(matrix, np.array(internal_indices), axis=1)

            ans[0:0] = internal_ans 

        if matrix.shape[0] == 0:
            return ans


variants = deepcopy(phones)

asc_dest = ascending_destilation(concordance_matrix, variants)

In [ ]:
for phone in asc_dest:
    print(phone)

PROMETHEE

In [ ]:
preference_matrix = np.zeros((len(phones), len(phones)))

for i, phone1 in enumerate(phones):
    for j, phone2 in enumerate(phones):
        pis = []
        for criterion in criteria:
            value1 = attribute_mapping[criterion.name](phone1)
            value2 = attribute_mapping[criterion.name](phone2)
            
            if criterion.type == "gain":
                if value1 - value2 >= criterion.pref:
                    pis.append(1)
                elif value1 - value2 <= criterion.indiff:
                    pis.append(0)
                else:
                    pis.append((value1 - value2 - criterion.indiff) / (criterion.pref - criterion.indiff))
            
            else:
                if value2 - value1 >= criterion.pref:
                    pis.append(1)
                elif value2 - value1 <= criterion.indiff:
                    pis.append(0)
                else:
                    pis.append((value2 - value1 - criterion.indiff) / (criterion.pref - criterion.indiff))
                    
        pi = sum(pi * criterion.weight for pi, criterion in zip(pis, criteria)) / sum(criterion.weight for criterion in criteria)

        preference_matrix[i][j] = round(pi, 2)

print(preference_matrix)

In [ ]:
positive_flow = np.sum(preference_matrix, axis=1).tolist()
negative_flow = np.sum(preference_matrix, axis=0).tolist()
net_flow = (np.array(positive_flow) - np.array(negative_flow)).tolist()

PROMETHEE I

In [ ]:
variants = deepcopy(phones)

combined = list(zip(positive_flow, variants))
combined.sort(key=lambda x: x[0], reverse=True)

positiv_ranking = [item[1] for item in combined]

combined = list(zip(negative_flow, variants))
combined.sort(key=lambda x: x[0])

negative_ranking = [item[1] for item in combined]

print("Positive ranking:")
for phone in positiv_ranking:
    print(phone.model)

print()

print("Negative ranking:")
for phone in negative_ranking:
    print(phone.model)

PROMETHEE II

In [ ]:
variants = deepcopy(phones)

combined = list(zip(net_flow, variants))
combined.sort(key=lambda x: x[0], reverse=True)

ranking = [item[1] for item in combined]

print("Final ranking:")
for phone in ranking:
    print(phone.model)